# Project Goal

The goal of this project was to conduct data analysis using primarily SQL, some Python, and a visualization tool Tableau  --> link in README file. 

The project focuses on the largest news publishers on YouTube, their activity and popularity. <br>
Ranking of the biggest news publishers on Youtube from October 2023:<br>
https://pressgazette.co.uk/media-audience-and-business-data/media_metrics/youtube-news-publishers-2023-gb-news-piers-morgan-cnn-fox/

Data was collected through the YouTube API.
I wanted to collect informations about videos from the last year 2023, but unfortunately I was unable to do so due to API limitations.

<b>Importing libraries</b>

In [1]:
from googleapiclient.discovery import build
import pandas as pd
import configparser
import json
import pymysql
from sqlalchemy import create_engine
import getpass 

<b>API key and initialization the API</b>

In [ ]:
# API key

config = configparser.ConfigParser()
config.read('secrets.txt')
api_key = config['API']['DEVELOPER_KEY']

youtube = build('youtube', 'v3', developerKey=api_key)

<b> Collecting the data</b> 

In [ ]:
# documentation:
# https://developers.google.com/youtube/v3/docs

In [ ]:
# id's of 20 biggest (english language) news publishers on Youtube
channel_ids = [
    'UCupvZG-5ko_eiXAupbDfxWw',
    'UCBi2mrWuNuyYy4gbM6fU18Q',
    'UC16niRr50-MSBwiO3YDb3RA',
    'UCNye-wNBqNL5ZzHSJj3l8Bg',
    'UCLXo7UDZvByw2ixzpQCufnA',
    'UCXIJgqnII2ZOINSWNOGFThA',
    'UCeY0bbntWzzVIaj2z3QigXg',
    'UCZaT_X_mc0BI-djXOlfhqWQ',
    'UCHJuQZuzapBh-CuhRYxIZrg',
    'UC_gUM8rL-Lrg6O3adPW9K1g',
    'UCoMdktPbSTixAyNGwb-UYkQ',
    'UCaXkIU1QidjPwiAYu6GcHjg',
    'UC1yBKRuGpC1tSM73A0ZjYjQ',
    'UC8p1vwvWtl6T73JiExfWs1g',
    'UCW39zufHfsuGgpLviKh297Q',
    'UCPgLNge0xqQHWM5B5EFH9Cg',
    'UCP6HGa63sBC7-KHtkme-p-g',
    'UCknLrEdhRCp1aegoMqRaCZg',
    'UCK7tptUDHh-RYDsdxO1-5QQ',
    'UCIzXayRP7-P0ANpq-nD-h5g'
]


def get_channel_data(youtube, channel_ids):
    all_channel_data = []

    request = youtube.channels().list(
        part="snippet,contentDetails,statistics,brandingSettings",
        id=','.join(channel_ids)
    )
    response = request.execute()

    for i in range(len(response['items'])):
        channel_data = dict(
            Channel_id=response['items'][i]['id'],
            Channel_name=response['items'][i]['snippet']['title'],
            Subscribers=response['items'][i]['statistics']['subscriberCount'],
            Views=response['items'][i]['statistics']['viewCount'],
            Total_videos=response['items'][i]['statistics']['videoCount'],
            Playlist_id=response['items'][i]['contentDetails']['relatedPlaylists']['uploads'],
            Description=response['items'][i]['snippet']['description'],
            Join_date=response['items'][i]['snippet']['publishedAt'],
            Playlist_count=response['items'][i]['statistics'].get('playlistCount', 0),
            Logo_url=response['items'][i]['brandingSettings']['image']['bannerExternalUrl']
        )
        all_channel_data.append(channel_data)

    return all_channel_data

def get_videos_data(youtube, playlist_id, channel_id, channel_name):
    all_video_data = []

    request = youtube.playlistItems().list(
        part='contentDetails',
        playlistId=playlist_id,
        maxResults=50
    )
    response = request.execute()

    for i in range(len(response['items'])):
        video_id = response['items'][i]['contentDetails']['videoId']
        video_stats = youtube.videos().list(
            part='snippet,statistics,contentDetails',
            id=video_id
        ).execute()

        video_data = dict(
            Channel_id=channel_id,
            Channel_name=channel_name,
            Video_id=video_id,
            Title=video_stats['items'][0]['snippet']['title'],
            Published_date=video_stats['items'][0]['snippet']['publishedAt'],
            Views=video_stats['items'][0]['statistics'].get('viewCount', 0),
            Likes=video_stats['items'][0]['statistics'].get('likeCount', 0),
            Dislikes=video_stats['items'][0]['statistics'].get('dislikeCount', 0),
            Comments=video_stats['items'][0]['statistics'].get('commentCount', 0),
            Tags=video_stats['items'][0]['snippet'].get('tags', []),
            Category=video_stats['items'][0]['snippet'].get('categoryId', 0),
            Duration=video_stats['items'][0]['contentDetails'].get('duration', '0'),
            Video_url=f"https://www.youtube.com/watch?v={video_id}"
        )
        all_video_data.append(video_data)

    return all_video_data

# getting channels data and creating DataFrame
channel_data = get_channel_data(youtube, channel_ids)
channel_df = pd.DataFrame(channel_data)

# getting videos data and creating DataFrame
video_data = []
for index, row in channel_df.iterrows():
    videos_data = get_videos_data(youtube, row['Playlist_id'], row['Channel_id'], row['Channel_name'])
    video_data.extend(videos_data)

video_df = pd.DataFrame(video_data)

# converting columns to appropriate types
channel_df['Join_date'] = pd.to_datetime(channel_df['Join_date']).dt.date
channel_df['Subscribers'] = pd.to_numeric(channel_df['Subscribers'])
channel_df['Views'] = pd.to_numeric(channel_df['Views'])
channel_df['Total_videos'] = pd.to_numeric(channel_df['Total_videos'])
channel_df['Playlist_count'] = pd.to_numeric(channel_df['Playlist_count'])

video_df['Published_date'] = pd.to_datetime(video_df['Published_date']).dt.date
video_df['Views'] = pd.to_numeric(video_df['Views'])
video_df['Likes'] = pd.to_numeric(video_df['Likes'])
video_df['Dislikes'] = pd.to_numeric(video_df['Dislikes'])
video_df['Comments'] = pd.to_numeric(video_df['Comments'])
video_df['Duration'] = pd.to_timedelta(video_df['Duration'])


#print("Channel DataFrame:")
#print(channel_df)

#print("\nVideo DataFrame:")
#print(video_df)

In [ ]:
# I tried to get more videos, but unfortunately I wasn't able to due to YT API restrictions :( 

<b> DataFrame -  channel_df :</b>

In [3]:
channel_df.head()

,Channel_id,Channel_name,Subscribers,Views,Total_videos,Playlist_id,Description,Join_date,Playlist_count,Logo_url
0,UCW39zufHfsuGgpLviKh297Q,DW Documentary,5240000,750795057,1049,UUW39zufHfsuGgpLviKh297Q,DW Documentary gives you information beyond th...,2016-11-22,0,https://yt3.googleusercontent.com/vJNY8qRXwLyF...
1,UCZaT_X_mc0BI-djXOlfhqWQ,VICE News,8760000,3123891725,6465,UUZaT_X_mc0BI-djXOlfhqWQ,The most important news of the day with contex...,2013-11-20,0,https://yt3.googleusercontent.com/SKEjS1GizIBz...
2,UCBi2mrWuNuyYy4gbM6fU18Q,ABC News,15600000,13806334136,84672,UUBi2mrWuNuyYy4gbM6fU18Q,ABC News is your daily source for breaking nat...,2006-08-07,0,https://yt3.googleusercontent.com/AQwcA9iETxA5...
3,UCIzXayRP7-P0ANpq-nD-h5g,The Sun,4630000,4439194754,19843,UUIzXayRP7-P0ANpq-nD-h5g,"Get the latest news, exclusives, sport, celebr...",2007-06-12,0,https://yt3.googleusercontent.com/N8OxOQEm3Fiu...
4,UCHJuQZuzapBh-CuhRYxIZrg,Insider,8460000,4940359244,7825,UUHJuQZuzapBh-CuhRYxIZrg,What you want to know. \n\nVisit our homepage ...,2015-09-25,0,https://yt3.googleusercontent.com/eEkYZCOAIqJ7...


In [4]:
channel_df.shape

(20, 10)

In [5]:
channel_df.isna().sum()

Channel_id        0
Channel_name      0
Subscribers       0
Views             0
Total_videos      0
Playlist_id       0
Description       0
Join_date         0
Playlist_count    0
Logo_url          0
dtype: int64

In [6]:
channel_df.dtypes

Channel_id        object
Channel_name      object
Subscribers        int64
Views              int64
Total_videos       int64
Playlist_id       object
Description       object
Join_date         object
Playlist_count     int64
Logo_url          object
dtype: object

In [7]:
channel_df['Subscribers'] = pd.to_numeric(channel_df['Subscribers'])
channel_df['Views'] = pd.to_numeric(channel_df['Views'])
channel_df['Total_videos'] = pd.to_numeric(channel_df['Total_videos'])
channel_df['Playlist_count'] = pd.to_numeric(channel_df['Playlist_count'])

In [8]:
channel_df.nunique()

Channel_id        20
Channel_name      20
Subscribers       20
Views             20
Total_videos      20
Playlist_id       20
Description       20
Join_date         20
Playlist_count     1
Logo_url          20
dtype: int64

In [9]:
channel_df.Playlist_count

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
Name: Playlist_count, dtype: int64

In [10]:
channel_df = channel_df.drop('Playlist_count', axis=1)

In [11]:
# columns names to snake case

channel_df.columns = channel_df.columns.str.lower().str.replace(' ', '_')

In [12]:
channel_df.columns

Index(['channel_id', 'channel_name', 'subscribers', 'views', 'total_videos',
       'playlist_id', 'description', 'join_date', 'logo_url'],
      dtype='object')

<b> DataFrame -  video_df :</b>

In [13]:
video_df.head()

,Channel_id,Channel_name,Video_id,Title,Published_date,Views,Likes,Dislikes,Comments,Tags,Category,Duration,Video_url
0,UCW39zufHfsuGgpLviKh297Q,DW Documentary,56YAjz2TxoA,Mysteries of the Inca: New insights in the And...,2023-12-17,87744,2079,0,156,"['Documentary', 'DW documentary', 'full docume...",27,0 days 01:25:56,https://www.youtube.com/watch?v=56YAjz2TxoA
1,UCW39zufHfsuGgpLviKh297Q,DW Documentary,-ERdZBpQ2mA,Leaving motherland? Migrating vs. staying put ...,2023-12-16,23518,351,0,83,"['Documentary', 'DW documentary', 'full docume...",27,0 days 00:28:31,https://www.youtube.com/watch?v=-ERdZBpQ2mA
2,UCW39zufHfsuGgpLviKh297Q,DW Documentary,faYdJYveX2w,Brazil: Life in Rio’s biggest favela | DW Docu...,2023-12-15,107519,1797,0,214,"['Documentary', 'DW documentary', 'full docume...",27,0 days 00:42:26,https://www.youtube.com/watch?v=faYdJYveX2w
3,UCW39zufHfsuGgpLviKh297Q,DW Documentary,0coUiuG5gG4,The global debt crisis - Is the world on the b...,2023-12-14,255768,3910,0,732,"['Documentary', 'DW documentary', 'full docume...",27,0 days 00:51:56,https://www.youtube.com/watch?v=0coUiuG5gG4
4,UCW39zufHfsuGgpLviKh297Q,DW Documentary,mzSxHe-hRho,By train through Vietnam - From Hanoi to Ho Ch...,2023-12-13,221742,3477,0,440,"['Documentary', 'DW documentary', 'full docume...",27,0 days 00:42:26,https://www.youtube.com/watch?v=mzSxHe-hRho


In [14]:
video_df.shape

(1000, 13)

In [15]:
video_df.isna().sum()

Channel_id        0
Channel_name      0
Video_id          0
Title             0
Published_date    0
Views             0
Likes             0
Dislikes          0
Comments          0
Tags              0
Category          0
Duration          0
Video_url         0
dtype: int64

In [16]:
video_df.dtypes

Channel_id        object
Channel_name      object
Video_id          object
Title             object
Published_date    object
Views              int64
Likes              int64
Dislikes           int64
Comments           int64
Tags              object
Category           int64
Duration          object
Video_url         object
dtype: object

In [17]:
video_df['Views'] = pd.to_numeric(video_df['Views'])
video_df['Likes'] = pd.to_numeric(video_df['Likes'])
video_df['Dislikes'] = pd.to_numeric(video_df['Dislikes'])
video_df['Comments'] = pd.to_numeric(video_df['Comments'])
video_df['Category'] = pd.to_numeric(video_df['Category'])
video_df['Published_date'] = pd.to_datetime(video_df['Published_date'])

In [18]:
video_df_without_tags = video_df.drop(columns=['Tags'])
unique_counts = video_df_without_tags.nunique()
unique_counts

Channel_id          20
Channel_name        20
Video_id          1000
Title              999
Published_date      89
Views              990
Likes              774
Dislikes             1
Comments           573
Category             8
Duration           556
Video_url         1000
dtype: int64

In [19]:
video_df.Dislikes.count

<bound method Series.count of 0      0
1      0
2      0
3      0
4      0
      ..
995    0
996    0
997    0
998    0
999    0
Name: Dislikes, Length: 1000, dtype: int64>

In [20]:
video_df = video_df.drop('Dislikes', axis=1)

In [21]:
# columns names to snake case

video_df.columns = video_df.columns.str.lower().str.replace(' ', '_')

In [22]:
video_df.columns

Index(['channel_id', 'channel_name', 'video_id', 'title', 'published_date',
       'views', 'likes', 'comments', 'tags', 'category', 'duration',
       'video_url'],
      dtype='object')

In [23]:
# category names from the json file

with open("category_id.json") as f: categoryID = json.load(f)
    
categoryID, l_c = categoryID['items'], len(categoryID['items'])

id_to_cat = dict(zip([int(categoryID[i]['id']) for i in range(l_c)],[categoryID[i]['snippet']['title'] for i in range(l_c)]))

In [24]:
id_to_cat.keys()

dict_keys([1, 2, 10, 15, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44])

In [25]:
sorted(video_df['category'].unique())

[17, 19, 22, 24, 25, 26, 27, 29]

In [26]:
# creating a new column with the category names

video_df['category_name'] = video_df['category'].apply(lambda x: id_to_cat[x])

# Saving Data 

<b>Saving to csv</b>

In [ ]:
channel_df.to_csv('clean_channels_top_news.csv', index=False)
video_df.to_csv('clean_videos_top_news.csv', index=False)

<b>Connecting to SQL</b>

In [27]:
# creating a connection to the SQL database
password = getpass.getpass("Please enter the SQL database password: ")
connection_string = 'mysql+pymysql://root:' + password + '@localhost/Ironhack_Final_Project'
engine = create_engine(connection_string)

# saving channel_df to SQL
with engine.connect() as connection:
    channel_df.to_sql(name='news_channels', con=connection, if_exists='append', index=False)

# saving video_df to SQL
with engine.connect() as connection:
    video_df.to_sql(name='news_videos', con=connection, if_exists='append', index=False)


Please enter the SQL database password: ········


# Analysis in SQL

<b>If you want to see the visualization of the analysis - please visit Tableau (link in README file)</b>

In [29]:
query_all_channels = "SELECT * FROM news_channels"
result_all_channels = pd.read_sql_query(query_all_channels, engine)
result_all_channels.head(3)

,channel_id,channel_name,subscribers,views,total_videos,playlist_id,description,join_date,logo_url
0,UCNye-wNBqNL5ZzHSJj3l8Bg,Al Jazeera English,12500000,3984315240,109723,UUNye-wNBqNL5ZzHSJj3l8Bg,"#AlJazeeraEnglish, we focus on people and even...",2006-11-23T14:12:43Z,https://yt3.googleusercontent.com/B3dF3C0Jyr8d...
1,UCZaT_X_mc0BI-djXOlfhqWQ,VICE News,8760000,3123891725,6465,UUZaT_X_mc0BI-djXOlfhqWQ,The most important news of the day with contex...,2013-11-20T15:11:51Z,https://yt3.googleusercontent.com/SKEjS1GizIBz...
2,UC1yBKRuGpC1tSM73A0ZjYjQ,The Young Turks,5690000,6527385660,57294,UU1yBKRuGpC1tSM73A0ZjYjQ,The Largest Online News Show in the World. Hos...,2005-12-21T20:46:51Z,https://yt3.googleusercontent.com/Z7JESOhtulMp...


In [30]:
query_all_channels = "SELECT * FROM news_videos"
result_all_channels = pd.read_sql_query(query_all_channels, engine)
result_all_channels.head(3)

,channel_id,channel_name,video_id,title,published_date,views,likes,comments,tags,category,duration,video_url,category_name
0,UCW39zufHfsuGgpLviKh297Q,DW Documentary,56YAjz2TxoA,Mysteries of the Inca: New insights in the And...,2023-12-17,87744,2079,156,"Documentary, DW documentary, full documentary,...",27,5156000000000,https://www.youtube.com/watch?v=56YAjz2TxoA,Education
1,UCW39zufHfsuGgpLviKh297Q,DW Documentary,-ERdZBpQ2mA,Leaving motherland? Migrating vs. staying put ...,2023-12-16,23518,351,83,"Documentary, DW documentary, full documentary,...",27,1711000000000,https://www.youtube.com/watch?v=-ERdZBpQ2mA,Education
2,UCW39zufHfsuGgpLviKh297Q,DW Documentary,faYdJYveX2w,Brazil: Life in Rio’s biggest favela | DW Docu...,2023-12-15,107519,1797,214,"Documentary, DW documentary, full documentary,...",27,2546000000000,https://www.youtube.com/watch?v=faYdJYveX2w,Education


In [31]:
# Total number of channels
total_channels_query = "SELECT COUNT(*) AS total_channels FROM news_channels"
total_channels = pd.read_sql_query(total_channels_query, engine)
print("Total number of channels:", total_channels['total_channels'].iloc[0])

Total number of channels: 20


In [32]:
# Average subscribers per channel
avg_subscribers_query = "SELECT AVG(subscribers) AS avg_subscribers FROM news_channels"
avg_subscribers = pd.read_sql_query(avg_subscribers_query, engine)
print("Average subscribers per channel:", avg_subscribers['avg_subscribers'].iloc[0])

Average subscribers per channel: 8620500.0


In [33]:
# Channels with the highest number of subscribers
top_channels_query = "SELECT channel_name, subscribers FROM news_channels ORDER BY subscribers DESC LIMIT 5"
top_channels = pd.read_sql_query(top_channels_query, engine)
print("Channels with the highest subscribers:")
top_channels

Channels with the highest subscribers:


,channel_name,subscribers
0,CNN,15900000
1,ABC News,15600000
2,BBC News,15300000
3,Al Jazeera English,12500000
4,Vox,11700000


In [34]:
# Channels with most videos
largest_playlist_query = """
SELECT channel_name, total_videos, playlist_id
FROM news_channels
ORDER BY total_videos DESC
LIMIT 5
"""
largest_playlist_results = pd.read_sql_query(largest_playlist_query, engine)
print("Channels with the largest playlist (by total videos):")
largest_playlist_results

Channels with the largest playlist (by total videos):


,channel_name,total_videos,playlist_id
0,CNN,162628,UUupvZG-5ko_eiXAupbDfxWw
1,WION,128262,UU_gUM8rL-Lrg6O3adPW9K1g
2,CBS News,117068,UU8p1vwvWtl6T73JiExfWs1g
3,Al Jazeera English,109723,UUNye-wNBqNL5ZzHSJj3l8Bg
4,Fox News,99870,UUXIJgqnII2ZOINSWNOGFThA


In [35]:
# Channels joined after a date
joined_before_query = "SELECT * FROM news_channels WHERE join_date > '2010-01-01'"
channels_joined_before = pd.read_sql_query(joined_before_query, engine)
print("Channels joined after date: 2010-01-01")
channels_joined_before

Channels joined after date: 2010-01-01


,channel_id,channel_name,subscribers,views,total_videos,playlist_id,description,join_date,logo_url
0,UCZaT_X_mc0BI-djXOlfhqWQ,VICE News,8760000,3123891725,6465,UUZaT_X_mc0BI-djXOlfhqWQ,The most important news of the day with contex...,2013-11-20T15:11:51Z,https://yt3.googleusercontent.com/SKEjS1GizIBz...
1,UCW39zufHfsuGgpLviKh297Q,DW Documentary,5240000,750795057,1049,UUW39zufHfsuGgpLviKh297Q,DW Documentary gives you information beyond th...,2016-11-22T12:18:08Z,https://yt3.googleusercontent.com/vJNY8qRXwLyF...
2,UCaXkIU1QidjPwiAYu6GcHjg,MSNBC,6300000,11623357709,75937,UUaXkIU1QidjPwiAYu6GcHjg,The official MSNBC YouTube Channel. MSNBC is ...,2011-12-01T19:16:09Z,https://yt3.googleusercontent.com/4ROAKx79gKHY...
3,UC_gUM8rL-Lrg6O3adPW9K1g,WION,8280000,4532857167,128262,UU_gUM8rL-Lrg6O3adPW9K1g,"WION -The World is One News, examines global i...",2016-05-19T12:55:08Z,https://yt3.googleusercontent.com/BmbJy-InvsFW...
4,UCLXo7UDZvByw2ixzpQCufnA,Vox,11700000,3382870754,1667,UULXo7UDZvByw2ixzpQCufnA,Vox helps you understand our world.\n\nVox vid...,2014-03-04T20:30:22Z,https://yt3.googleusercontent.com/y4c_KyXDPmHL...
5,UCHJuQZuzapBh-CuhRYxIZrg,Insider,8460000,4940359244,7825,UUHJuQZuzapBh-CuhRYxIZrg,What you want to know. \n\nVisit our homepage ...,2015-09-25T19:37:39Z,https://yt3.googleusercontent.com/eEkYZCOAIqJ7...


In [36]:
join_duration_query = """
SELECT channel_name, join_date, NOW() - join_date AS join_duration
FROM news_channels
ORDER BY join_duration DESC
LIMIT 5
"""
join_duration_results = pd.read_sql_query(join_duration_query, engine)
print("Channels with the longest join duration:")
join_duration_results

Channels with the longest join duration:


,channel_name,join_date,join_duration
0,The Young Turks,2005-12-21T20:46:51Z,2.023122e+13
1,CNN,2005-10-02T16:06:36Z,2.023122e+13
2,Al Jazeera English,2006-11-23T14:12:43Z,2.023122e+13
3,The Telegraph,2006-11-19T00:03:44Z,2.023122e+13
4,BBC News,2006-04-08T05:51:05Z,2.023122e+13


In [37]:
description_length_query = """
SELECT channel_name, description
FROM news_channels
WHERE LENGTH(description) > 100
LIMIT 5
"""
description_length_results = pd.read_sql_query(description_length_query, engine)
print("Channels with descriptions longer than 100 characters:")
description_length_results

Channels with descriptions longer than 100 characters:


,channel_name,description
0,Al Jazeera English,"#AlJazeeraEnglish, we focus on people and even..."
1,VICE News,The most important news of the day with contex...
2,The Young Turks,The Largest Online News Show in the World. Hos...
3,The Telegraph,"The latest UK and world news videos, opinion p..."
4,BBC News,Welcome to the official BBC News YouTube chann...


In [38]:
views_per_subscriber_query = """
SELECT channel_name, subscribers, views, (views/subscribers) AS views_per_subscriber
FROM news_channels
ORDER BY views_per_subscriber DESC
LIMIT 5
"""
views_per_subscriber_results = pd.read_sql_query(views_per_subscriber_query, engine)
print("Channels with the highest views per subscriber:")
views_per_subscriber_results

Channels with the highest views per subscriber:


,channel_name,subscribers,views,views_per_subscriber
0,MSNBC,6300000,11623357709,1844.9774
1,Fox News,10900000,15812179598,1450.6587
2,The Young Turks,5690000,6527385660,1147.1680
3,The Sun,4630000,4439194754,958.7894
4,CNN,15900000,15086195585,948.8173


In [39]:
videos_to_subscribers_ratio_query = """
SELECT channel_name, subscribers, total_videos, (total_videos/subscribers) AS videos_to_subscribers_ratio
FROM news_channels
ORDER BY videos_to_subscribers_ratio DESC
LIMIT 5
"""
videos_to_subscribers_ratio_results = pd.read_sql_query(videos_to_subscribers_ratio_query, engine)
print("\nChannels with a high ratio of total videos to subscribers:")
videos_to_subscribers_ratio_results


Channels with a high ratio of total videos to subscribers:


,channel_name,subscribers,total_videos,videos_to_subscribers_ratio
0,CBS News,5500000,117068,0.0213
1,WION,8280000,128262,0.0155
2,MSNBC,6300000,75937,0.0121
3,CNN,15900000,162628,0.0102
4,The Young Turks,5690000,57294,0.0101


In [40]:
recent_videos_query = """
SELECT c.channel_name, v.title, v.published_date
FROM news_channels c
JOIN news_videos v ON c.channel_id = v.channel_id
ORDER BY v.published_date DESC
LIMIT 5
"""
recent_videos_results = pd.read_sql_query(recent_videos_query, engine)
print("Channels with the most recently uploaded videos:")
recent_videos_results

Channels with the most recently uploaded videos:


,channel_name,title,published_date
0,Al Jazeera English,2023 in Review: Drones have shaped the Ukraine...,2023-12-18
1,Sky News,Business Live with Ian King: UK to launch carb...,2023-12-18
2,ABC News,Israeli Prime Minister Benjamin Netanyahu says...,2023-12-18
3,ABC News,Cars seen floating in street as South Carolina...,2023-12-18
4,ABC News,American and British warship shot down 15 dron...,2023-12-18


In [41]:
engagement_query = """
SELECT c.channel_name, c.subscribers, SUM(v.likes) AS total_likes,
       (SUM(v.likes)/c.subscribers) AS engagement
FROM news_channels c
JOIN news_videos v ON c.channel_id = v.channel_id
GROUP BY c.channel_name, c.subscribers
ORDER BY engagement DESC
LIMIT 5
"""
engagement_results = pd.read_sql_query(engagement_query, engine)
print("Channels with the highest engagement (total likes per subscriber):")
engagement_results

Channels with the highest engagement (total likes per subscriber):


,channel_name,subscribers,total_likes,engagement
0,Vox,11700000,1939516.0,0.1658
1,The Wall Street Journal,4870000,206225.0,0.0423
2,DW Documentary,5240000,171868.0,0.0328
3,Insider,8460000,244207.0,0.0289
4,The Young Turks,5690000,144395.0,0.0254


In [42]:
average_views_query = """
SELECT c.channel_name, AVG(v.views) AS avg_views_per_video
FROM news_channels c
JOIN news_videos v ON c.channel_id = v.channel_id
GROUP BY c.channel_name
ORDER BY avg_views_per_video DESC
LIMIT 5
"""
average_views_results = pd.read_sql_query(average_views_query, engine)
print("Channels with the highest average views per video:")
average_views_results

Channels with the highest average views per video:


,channel_name,avg_views_per_video
0,Vox,868729.04
1,Insider,306746.74
2,The Wall Street Journal,303175.68
3,DW Documentary,270904.56
4,VICE News,212732.76


In [43]:
# Videos with the Highest Views
highest_views_query = """
SELECT title, views, likes, comments
FROM news_videos
ORDER BY views DESC
LIMIT 5
"""
highest_views_results = pd.read_sql_query(highest_views_query, engine)
print("Videos with the highest number of views:")
highest_views_results

Videos with the highest number of views:


,title,views,likes,comments
0,Something weird happens when you keep squeezing,6222575,142820,4604
1,"The Hesitant Fiancée, explained in 60 seconds ...",6172946,605958,2184
2,Special Ops Sniper Rates 11 More Sniper Scenes...,6008189,74807,2387
3,"The rise of Xi Jinping, explained",3438409,96809,4405
4,North Korea's most powerful woman | DW Documen...,3255550,33995,4773


In [44]:
# Videos with the Most Likes
most_likes_query = """
SELECT title, likes, views, comments
FROM news_videos
ORDER BY likes DESC
LIMIT 5
"""
most_likes_results = pd.read_sql_query(most_likes_query, engine)
print("\nVideos with the most likes:")
most_likes_results


Videos with the most likes:


,title,likes,views,comments
0,"The Hesitant Fiancée, explained in 60 seconds ...",605958,6172946,2184
1,Something weird happens when you keep squeezing,142820,6222575,4604
2,What's inside this crater in Madagascar?,115980,2734814,6174
3,Popular names aren’t as popular as they used t...,101826,1628420,2258
4,Why this 13th-century song shows up everywhere...,98244,1042102,1306


In [45]:
# Videos with the Most Comments
most_comments_query = """
SELECT title, comments, views, likes
FROM news_videos
ORDER BY comments DESC
LIMIT 5
"""
most_comments_results = pd.read_sql_query(most_comments_query, engine)
print("\nVideos with the most comments:")
most_comments_results


Videos with the most comments:


,title,comments,views,likes
0,Cenk Goes On RAMPAGE About Attacks on Harvard ...,14885,465316,13476
1,"Gaza, explained",12578,3013163,91314
2,Right-Wingers Cheer Man Who BEHEADED Satanic T...,8161,144467,3454
3,What's inside this crater in Madagascar?,6174,2734814,115980
4,Hungary blocks €50bn of EU funding for Ukraine...,5860,395554,9787


In [46]:
# Longest Videos in Dataframe
longest_videos_query = """
SELECT title, duration, views, likes, comments
FROM news_videos
ORDER BY duration DESC
LIMIT 5
"""
longest_videos_results = pd.read_sql_query(longest_videos_query, engine)
print("Longest videos:")
longest_videos_results

Longest videos:


,title,duration,views,likes,comments
0,"LIVE: ABC News Live - Friday, December 15 | AB...",236392000000000,119142,893,0
1,LIVE: View over Israel-Gaza border as strikes ...,32337000000000,18833,243,14
2,In full: Rishi Sunak’s Rwanda Bill passes desp...,24785000000000,18595,109,32
3,Russia-Ukraine war LIVE: Russia shot down nume...,20371000000000,4964,54,0
4,Israel-Hamas latest: Netanyahu vows to continu...,17956000000000,28971,196,49


In [47]:
# Number of Videos in Each Category for a Channel
videos_per_category_query = """
SELECT c.channel_name, v.category_name, COUNT(v.video_id) AS num_videos_in_category
FROM news_channels c
JOIN news_videos v ON c.channel_id = v.channel_id
GROUP BY c.channel_name, v.category_name
ORDER BY c.channel_name, num_videos_in_category DESC
"""
videos_per_category_results = pd.read_sql_query(videos_per_category_query, engine)
print("\nNumber of videos in each category for a channel (in this Dataframe):")
videos_per_category_results


Number of videos in each category for a channel (in this Dataframe):


,channel_name,category_name,num_videos_in_category
0,ABC News,News & Politics,50
1,Al Jazeera English,News & Politics,50
2,BBC News,News & Politics,50
3,CBS News,News & Politics,50
4,CNN,News & Politics,50
5,DW Documentary,Education,50
6,DW News,News & Politics,50
7,Fox News,News & Politics,50
8,Insider,Entertainment,34
9,Insider,People & Blogs,15


In [48]:
most_videos_category_query = """
SELECT category_name, COUNT(video_id) AS num_videos
FROM news_videos
GROUP BY category_name
ORDER BY num_videos DESC
LIMIT 1
"""
most_videos_category_result = pd.read_sql_query(most_videos_category_query, engine)
print("Category with the most number of videos:")
most_videos_category_result

Category with the most number of videos:


,category_name,num_videos
0,News & Politics,874


In [49]:
highest_likes_category_query = """
SELECT category_name, AVG(likes) AS avg_likes_per_video
FROM news_videos
GROUP BY category_name
ORDER BY avg_likes_per_video DESC
LIMIT 1
"""
highest_likes_category_result = pd.read_sql_query(highest_likes_category_query, engine)
print("Category with the highest average likes per video:")
highest_likes_category_result

Category with the highest average likes per video:


,category_name,avg_likes_per_video
0,Entertainment,4385.8889


In [50]:
max_date_query = """
SELECT MAX(published_date) AS max_published_date
FROM news_videos
"""
max_date_result = pd.read_sql_query(max_date_query, engine)
print("Maximum published date in news_videos (in this DataFrame):")
max_date_result

Maximum published date in news_videos (in this DataFrame):


,max_published_date
0,2023-12-18


In [51]:
max_date_with_video_query = """
SELECT *
FROM news_videos
WHERE published_date = (SELECT MAX(published_date) FROM news_videos)
LIMIT 5
"""
max_date_with_video_result = pd.read_sql_query(max_date_with_video_query, engine)
print("Videos with the maximum published date (in this DataFrame):")
max_date_with_video_result

Videos with the maximum published date (in this DataFrame):


,channel_id,channel_name,video_id,title,published_date,views,likes,comments,tags,category,duration,video_url,category_name
0,UCBi2mrWuNuyYy4gbM6fU18Q,ABC News,XY8Jw9MyUGw,Matthew Perry died from acute effects of ketam...,2023-12-18,1624,26,7,"abc, angeles, autopsy, cause, death, friends, ...",25,326000000000,https://www.youtube.com/watch?v=XY8Jw9MyUGw,News & Politics
1,UCBi2mrWuNuyYy4gbM6fU18Q,ABC News,oZD8sdd6g7E,Severe weather causes travel trouble ahead of ...,2023-12-18,434,8,1,"abc, airlines, buttigieg, cancellations, flyin...",25,148000000000,https://www.youtube.com/watch?v=oZD8sdd6g7E,News & Politics
2,UCBi2mrWuNuyYy4gbM6fU18Q,ABC News,b8j0ASAuVUs,Sandra Day O'Connor's casket arrives at Suprem...,2023-12-18,690,27,4,"abc, court, funeral, honored, news, o'connor, ...",25,504000000000,https://www.youtube.com/watch?v=b8j0ASAuVUs,News & Politics
3,UCBi2mrWuNuyYy4gbM6fU18Q,ABC News,k5t59vrkxK0,Netanyahu vows to press on with Gaza offensive,2023-12-18,1204,14,50,"abc, defense, gaza, gma, hamas, international,...",25,180000000000,https://www.youtube.com/watch?v=k5t59vrkxK0,News & Politics
4,UCBi2mrWuNuyYy4gbM6fU18Q,ABC News,W6Vc2GA5xSk,A first look at “Daughters of the Cult”,2023-12-18,672,24,8,"news, breaking news, live news, daily news, wo...",25,43000000000,https://www.youtube.com/watch?v=W6Vc2GA5xSk,News & Politics


In [52]:
min_date_query = """
SELECT MIN(published_date) AS min_published_date
FROM news_videos
"""
min_date_result = pd.read_sql_query(min_date_query, engine)
print("\nMinimum published date in news_videos (in this DataFrame):")
min_date_result


Minimum published date in news_videos (in this DataFrame):


,min_published_date
0,2023-08-28


In [53]:
min_date_with_video_query = """
SELECT *
FROM news_videos
WHERE published_date = (SELECT MIN(published_date) FROM news_videos)
LIMIT 5
"""
min_date_with_video_result = pd.read_sql_query(min_date_with_video_query, engine)
print("\nVideos with the minimum published date (in this DataFrame):")
min_date_with_video_result


Videos with the minimum published date (in this DataFrame):


,channel_id,channel_name,video_id,title,published_date,views,likes,comments,tags,category,duration,video_url,category_name
0,UCLXo7UDZvByw2ixzpQCufnA,Vox,6xvrKW2H_hA,Why Silicon Valley is here,2023-08-28,766339,18456,369,"Palo Alto, Stanford, Vox.com, apple park, expl...",25,527000000000,https://www.youtube.com/watch?v=6xvrKW2H_hA,News & Politics
